This notebook is used to generate the densities using only the final max_max set, to compare with the max set. Remember that the max max set is calculating only one community for every t for every algorithm. Is used to modify the final normalized set calculation.  (so, instead of having a matrix of HxM, you only have an arry of size M)

In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib
import scipy.io as sio
import mne 
import os
from sklearn.metrics import roc_curve, auc
from functions.preprocessing import *

In [50]:
patients = [
'sub-HUP139',
'sub-HUP180',
'sub-HUP146',
'sub-HUP177',
'sub-HUP185',
'sub-HUP117',
'sub-HUP173',
'sub-HUP160',
'sub-HUP148',
'sub-HUP150',
'sub-HUP130',
'sub-HUP141',
'sub-HUP144',
'sub-HUP164',
'sub-HUP142',
'sub-HUP172',
'sub-HUP151',
'sub-HUP171',
'sub-HUP187',
] 

# patients=['S1','S2','S3','S4','S5']
percentiles=[
            0.9,
            0.95,
            0.99
            ]
ref_data=0
bands = [
        'alpha',
         'beta',
         'theta',
         'full_gamma',
         'low_gamma',
         'high_gamma1'
         ]
method_exp = 'aec'
norm = [
        '',
        'distance_'
        ]
experiment=''


output_path="/home/pablo/works/dev_thesis_SEEG/outputs_t60/"
main_path='/home/pablo/works/dev_thesis_SEEG/data/mainDatabase_patients/'
# main_path='/home/pablo/works/dev_thesis_SEEG/data/TVB_patients'
# output_path="/home/pablo/works/dev_thesis_SEEG/outputs_simil/"


In [ ]:
for patient in patients: 
    doc_file_data=pd.read_csv(main_path+'/'+patient+'/'+'ses-presurgery/'+patient+'_ses-presurgery_scans.tsv',sep='\t')
    raw=mne.io.read_raw_edf(main_path+'/'+patient+'/'+'ses-presurgery/'+doc_file_data['filename'][ref_data],preload=True)
    xyz_loc=pd.read_csv(main_path+'/'+patient+'/'+'ses-presurgery/ieeg/'+patient+'_ses-presurgery_acq-seeg_space-fsaverage_electrodes.tsv',sep='\t')
    events=pd.read_csv(main_path+'/'+patient+'/'+'ses-presurgery/'+doc_file_data['filename'][ref_data].replace('_ieeg.edf','_events.tsv'),sep='\t')
    channels=pd.read_csv(main_path+'/'+patient+'/'+'ses-presurgery/'+doc_file_data['filename'][ref_data].replace('_ieeg.edf','_channels.tsv'),sep='\t')
    raw, xyz_loc, inside_network= format_data_database(raw, xyz_loc, events=events, channels=channels)


    # raw=mne.io.read_raw_fif(main_path+'/'+patient+'/'+f'{patient}_{ref_data}.fif',preload=True)
    # xyz_loc=pd.read_csv(main_path+'/'+patient+'/'+'xyz_loc.csv',sep=',')
    # raw, xyz_loc, inside_network= format_data(raw, xyz_loc, events=None, channels=None)


    for percentile in percentiles:
        for n in norm:
            for band in bands:
                try:
                    with open(f'{output_path}{patient}/ref_{ref_data}/percentile_{percentile}/final_com_{band}_{method_exp}_{n}.json') as f:
                        final_communities_data = json.load(f)

                        # Step 1: Assign cumulative densities
                    contact_densities = {}
                    for community_type, community_list, density in final_communities_data:
                        for community in community_list:
                            for contact in community:
                                if contact not in contact_densities:
                                    contact_densities[contact] = 0
                                contact_densities[contact] += density
                    
                    

                    # Step 2: Normalize cumulative densities
                    max_density = max(contact_densities.values())
                    min_density = min(contact_densities.values())
                    if max_density == min_density:
                        if max_density == 0:
                            max_density = 1
                        else:
                            min_density = 0

                    normalized_densities = {k: (v - min_density) / (max_density - min_density) for k, v in contact_densities.items()}

                    #Step 3: If there is a key in xyz_loc['formatted_label] that is not in normalized_densities, add it with a value of 0
                    for key in xyz_loc['formatted_label']:
                        if key not in normalized_densities:
                            normalized_densities[key] = 0

                    df=pd.DataFrame.from_dict(normalized_densities,orient='index',columns=['Normalized density'])
                    # Save the normalized densities
                    df.to_csv(os.path.join(output_path, f'{patient}/ref_{ref_data}/percentile_{percentile}/normalized_densities_maxmax_{band}_{n}_{method_exp}.csv'))

                except Exception as e:
                    print()
                    print(f"Error processing {patient}, band {band}, norm {n}, percentile {percentile}: {e}")
                    # results_df.loc[(patient, percentile, n), band] = None
                    continue